In [99]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import cross_validate
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import seaborn as sns

In [100]:
#importing train data
train = pd.read_csv('/share/Datasets/Kaggle Titanic/train.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [101]:
#importing test datat
test = pd.read_csv('/share/Datasets/Kaggle Titanic/test.csv')
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [102]:
#checking if Sex coulmn has any data missing
train["Sex"].isnull().any()

False

In [103]:
#checking if Age coulmn has any data missing
train["Age"].isnull().any()

True

In [104]:
#Replacing the missing data in train and test set for Age column with their mean
agemean = train["Age"].mean()
train["Age"].fillna(value=agemean, inplace=True)
test["Age"].fillna(value=agemean, inplace=True)

In [105]:
#checking if Fare coulmn has any data missing
test["Fare"].isnull().any()

True

In [106]:
#Replacing the missing data in train and test set for Fare column with their median
# Median is used as the fares are likely skewed
faremean = train["Fare"].median()
train["Fare"].fillna(value=faremean, inplace=True)
test["Fare"].fillna(value=faremean, inplace=True)

In [107]:
# Pulling out data subsets for training and testing
features = train.loc[:,["Pclass","Sex","Age","SibSp","Parch","Fare"]]
targets = train.loc[:,"Survived"]
test_features = test.loc[:,["Pclass","Sex","Age","SibSp","Parch","Fare"]]
features.head()

,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,male,22.0,1,0,7.2500
1,1,female,38.0,1,0,71.2833
2,3,female,26.0,0,0,7.9250
3,1,female,35.0,1,0,53.1000
4,3,male,35.0,0,0,8.0500


In [108]:
#dummy coding Sex column to covert from categorial for training data
sex_dc = pd.get_dummies(features["Sex"])
featuresdc1 = pd.concat([features, sex_dc], axis=1)
featuresdc1.head()
featuresdc = featuresdc1.drop(columns = ["Sex"], axis = 1) 
featuresdc.head()



,Pclass,Age,SibSp,Parch,Fare,female,male
0,3,22.0,1,0,7.2500,0,1
1,1,38.0,1,0,71.2833,1,0
2,3,26.0,0,0,7.9250,1,0
3,1,35.0,1,0,53.1000,1,0
4,3,35.0,0,0,8.0500,0,1


In [109]:
#dummy coding Sex column to covert from categorial for testing data
sex_dc1 = pd.get_dummies(test_features["Sex"])
test_featuresdc1 = pd.concat([test_features, sex_dc1], axis=1)
test_featuresdc1.head()
test_featuresdc = test_featuresdc1.drop(columns = ["Sex"], axis = 1) 
test_featuresdc.head()


,Pclass,Age,SibSp,Parch,Fare,female,male
0,3,34.5,0,0,7.8292,0,1
1,3,47.0,1,0,7.0000,1,0
2,2,62.0,0,0,9.6875,0,1
3,3,27.0,0,0,8.6625,0,1
4,3,22.0,1,1,12.2875,1,0


In [110]:
#confirming where the Fare column is
featuresdc.iloc[:,4:5]

,Fare
0,7.2500
1,71.2833
2,7.9250
3,53.1000
4,8.0500
5,8.4583
6,51.8625
7,21.0750
8,11.1333
9,30.0708


In [111]:
#using K-means clustering for Fare column.
kmeans = KMeans(n_clusters=4, random_state=0).fit(featuresdc.iloc[:,4:5])
kmeans.cluster_centers_

array([[ 15.05337292],
       [204.94806429],
       [ 75.57133857],
       [512.3292    ]])

In [112]:
kmeans.labels_

array([0, 2, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 2, 0, 2, 2, 0, 0, 0,
       0, 0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       1, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0,
       2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0,
       0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0,
       0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 3, 0, 0, 0, 2, 0,
       0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 1, 0, 1, 0,

In [113]:
#adding the K-means clustering Fare data into training set
featuresdc['faredc'] = kmeans.labels_
featuresdc.head()

,Pclass,Age,SibSp,Parch,Fare,female,male,faredc
0,3,22.0,1,0,7.2500,0,1,0
1,1,38.0,1,0,71.2833,1,0,2
2,3,26.0,0,0,7.9250,1,0,0
3,1,35.0,1,0,53.1000,1,0,2
4,3,35.0,0,0,8.0500,0,1,0


In [114]:
#dropping the Fare column as it is no longer used
featuresdc = featuresdc.drop(columns = ["Fare"], axis = 1) 
featuresdc.head()

,Pclass,Age,SibSp,Parch,female,male,faredc
0,3,22.0,1,0,0,1,0
1,1,38.0,1,0,1,0,2
2,3,26.0,0,0,1,0,0
3,1,35.0,1,0,1,0,2
4,3,35.0,0,0,0,1,0


In [115]:
#dummy coding the new fare column data.
fare_dc = pd.get_dummies(featuresdc["faredc"])
featuresdc1 = pd.concat([featuresdc, fare_dc], axis=1)
featuresdc1.head()


,Pclass,Age,SibSp,Parch,female,male,faredc,0,1,2,3
0,3,22.0,1,0,0,1,0,1,0,0,0
1,1,38.0,1,0,1,0,2,0,0,1,0
2,3,26.0,0,0,1,0,0,1,0,0,0
3,1,35.0,1,0,1,0,2,0,0,1,0
4,3,35.0,0,0,0,1,0,1,0,0,0


In [116]:
#dropping the Fare_dc column as it is no longer used
featuresdc2 = featuresdc1.drop(columns = ["faredc"], axis = 1) 
featuresdc2.head()

,Pclass,Age,SibSp,Parch,female,male,0,1,2,3
0,3,22.0,1,0,0,1,1,0,0,0
1,1,38.0,1,0,1,0,0,0,1,0
2,3,26.0,0,0,1,0,1,0,0,0
3,1,35.0,1,0,1,0,0,0,1,0
4,3,35.0,0,0,0,1,1,0,0,0


In [117]:
#conducting the same process for testing data
kmeans = KMeans(n_clusters=4, random_state=0).fit(test_featuresdc.iloc[:,4:5])
kmeans.cluster_centers_
kmeans.labels_
test_featuresdc['faredc1'] = kmeans.labels_
test_featuresdc.head()
test_featuresdc = test_featuresdc.drop(columns = ["Fare"], axis = 1) 
test_featuresdc.head()
fare_dc1 = pd.get_dummies(test_featuresdc["faredc1"])
test_featuresdc2 = pd.concat([test_featuresdc, fare_dc1], axis=1)
test_featuresdc2.head()
test_featuresdc3 = test_featuresdc2.drop(columns = ["faredc1"], axis = 1) 
test_featuresdc3.head()

,Pclass,Age,SibSp,Parch,female,male,0,1,2,3
0,3,34.5,0,0,0,1,1,0,0,0
1,3,47.0,1,0,1,0,1,0,0,0
2,2,62.0,0,0,0,1,1,0,0,0
3,3,27.0,0,0,0,1,1,0,0,0
4,3,22.0,1,1,1,0,1,0,0,0


In [118]:
# Building a random forest classifier with a minimum samples leaf of 5
forest = RandomForestClassifier(n_estimators=40, min_samples_leaf=5)
scoring = ['accuracy','precision', 'recall', 'f1']
scores = cross_validate(forest, featuresdc2, targets, scoring=scoring, cv=5)
# Printing out the scores of interest
print("Accuracy", scores['test_accuracy'])
print("Precision", scores['test_precision'])
print("Recall", scores['test_recall'])
print("F1", scores['test_f1'])

Accuracy [0.78212291 0.83240223 0.83707865 0.79775281 0.83050847]
Precision [0.74193548 0.81967213 0.83050847 0.82       0.82758621]
Recall [0.66666667 0.72463768 0.72058824 0.60294118 0.70588235]
F1 [0.70229008 0.76923077 0.77165354 0.69491525 0.76190476]


In [121]:
# Building a random forest classifier with a minimum samples leaf of 20
forest1 = RandomForestClassifier(n_estimators=40, min_samples_leaf=20)
scoring = ['accuracy','precision', 'recall', 'f1']
scores = cross_validate(forest, featuresdc2, targets, scoring=scoring, cv=5)
# Printing out the scores of interest
print("Accuracy", scores['test_accuracy'])
print("Precision", scores['test_precision'])
print("Recall", scores['test_recall'])
print("F1", scores['test_f1'])

Accuracy [0.80446927 0.82681564 0.81460674 0.78089888 0.85310734]
Precision [0.765625   0.81666667 0.78688525 0.80851064 0.875     ]
Recall [0.71014493 0.71014493 0.70588235 0.55882353 0.72058824]
F1 [0.73684211 0.75968992 0.74418605 0.66086957 0.79032258]


In [122]:
# Finally applying the second model to predict values on the test set as we can see that the accuracy and precision in second 
#model is better than the first model we used.
fitmodel = forest.fit(featuresdc2, targets)
preds = fitmodel.predict(test_featuresdc3)
# Building a result set with the passengerid and the predicted survived column
result = pd.DataFrame(test["PassengerId"])
result["Survived"] = preds
result.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
